In [8]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

### State Variables
- Predator’s (Dragon) Weight in kg (W) 
- Predator’s Calories Required in kcal (C) 
- Prey Population (P)
- Dragon’s Height in cm (H)

### Input Variables
- Time (n) as a discrete variable, where n is the number of years since the birth of the dragon
- The state vector from the current timestep; this is a first order recurrence relation. 

### Output Variables
- The state vector for the next timestep 

### Parameters  
- s :  Scaling constant between calories and kilograms  

To convert the excess calories eaten (kcal) into kilograms of weight gained/losted, we will use a proportionality constant for mass conversion of 5/7700. 

- $\alpha$: Represents the number of calories per prey eaten

We will use the assumption that dragons are carnivore’s who, like Lions, consume horse meat to derive the number of calories per prey eaten.  Based upon research 1 pound of horsemeat contains 600 kcals and that each horse weighs on average 1400 pounds. If we assume that 30% of the horse’s weight contains horse meat then we can calculate the number of calories per prey from the following formula: $$600 \times 1400 \times 0.3 = 252000$$. 

- $\beta$: Represents the proportion of preys consumed by the predator in a year

We will initially assume that 5% of the prey population is consumed each year. 

- d: The activity level of a dragon 

This is an activity constant for the Harris-Benedict formula. Since dragons tend to fly large distances, we will assume that their activity level coefficient is 1.9 (which is used for ‘extremely active’ humans)

- b: Intrinsic growth rate for prey population

We will assume that the birth rate for the prey population is approximately 15%, when it has not reached its carrying capacity. 

- v: Carrying capacity of prey population 

We will assume that the carrying capacity will initially be 5000 for the prey population. 

- u : Proportionality Constant between Prey Population and Weight of Dragon

We will assume that the proportionality constant between the prey population and the weight of the dragon will be 0.0005. 

- r: The intrinsic growth rate for Height 

We need to specify the intrinsic growth rate for Height, which is essentially the rate at which we expect the height to increase over time before it reaches its stabilized height. Based upon the rate of height growth in the early childhood of humans, we will assume that the intrinsic growth rate is 0.5 (50%).   

- k: Stabilizing height in cm 

We will be modelling the height of a dragon using a logistic model, so we need to determine the value that the height of the dragon stabilizes at. We will initially assume that dragon’s cannot grow taller than 200cm. 


$$W(n+1) = W(n) - s(C(n)- (1/365)\alpha\beta P(n))$$

$$C(n+1) = d(10W(n) + 6.25H(n) - 5n - 161)$$ 

$$P(n+1) = (1 + b)(1 - \frac{P(n)}{v})P(n) + P(n) - (uP(n)W(n))$$

$$H(n+1) = H(n)(r(1 - \frac{H(n)}{k}) + 1)$$


In [316]:
# define parameters 

# equation 1
s = (5/7700) # proportionality constant for mass conversion  

alpha = 252000 # number of calories per prey eaten

beta = 0.05 # average percentage of population eaten per year


# equation 2
d = 1.9 # activity level constant


# equation 3
b = 0.15 # yearly birth rate for preys 

v = 1000 # carrying capacity of prey population 

u = 0.0005 # proportionality constant between weight and preys eaten


# equation 4

r = 0.15 # intrinsic growth rate for height 

k = 200 # stablizing height in cm 





def f(n, W, C, P, H):
    W_n = W - s*(C - (1/365 * alpha * beta * P))
    
    C_n = d * ((10 * W) + (6.25 * H) - (5 * n) - 161)
    
    P_n = (1 + b) * (1 - (P / v)) * P + P - (u * P * W) # use a logistic model
    
    H_n = H * (r * (1 - (H/k)) + 1)
    
    n += 1
    
    return n, W_n, C_n, P_n, H_n

In [317]:
def run_simulation(timestep, X_0):
    
    columns = ['n','W', 'C', 'P', 'H']
    
    df = pd.DataFrame(columns = columns, index = range(timestep + 1)) # create dataframe to store results
    
    df.iloc[0] = X_0
    
    for i in range(1 , timestep + 1):
        X_0 = f(*X_0)
        X_0 = list(X_0)
        for v in range(len(X_0)):
            if X_0[v] < 0:
                X_0[v] = 0
        
        X_0 = tuple(X_0)
        df.iloc[i] = X_0
    return df  

In [320]:
# intial state: n = 0, weight = 10kg, calories per day = 100, prey population = 1000, height  = 50cm
intial = (0, 10, 500, 950, 50) 
run_simulation(100, intial)

,n,W,C,P,H
0,0,10,500,950,50
1,1,30.970468,477.85,999.875,55.625
2,2,53.073314,933.585765,984.535434,61.648145
3,3,74.536377,1415.564681,975.91838,68.044996
4,4,95.493307,1889.825486,966.57462,74.779154
...,...,...,...,...,...
96,96,867.117582,17591.846152,624.593522,199.999873
97,97,869.695157,17632.332551,623.443432,199.999892
98,98,872.220661,17671.80669,622.316542,199.999908
99,99,874.695272,17710.291461,621.212333,199.999922
